## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [5]:
import numpy as np

In [9]:
import random

class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob
        
    def _generate_splits(self, data: np.ndarray):
        '''
        Generate indices for every bag and store in self.indices_list list
        '''
        self.indices_list = []
        data_length = len(data)
        for bag in range(self.num_bags):
            # Your Code Here
            # набор всевозможных индексов объектов
            possible_indexes = list(range(data_length))
            # генерируем наборы случайных индексов
            self.indices_list.append([random.choice(possible_indexes) for  _ in range(data_length)])
        
    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.
        
        example:
        
        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'
        self.models_list = []
        for bag in range(self.num_bags):
            model = model_constructor()
            # Для каждой модели формируем выборки объектов
            # выбираем объекты с индексами в соответствующем indices_list
            data_bag, target_bag = [data[idx] for idx in self.indices_list[bag]], [target[idx] for idx in self.indices_list[bag]] # Your Code Here
            self.models_list.append(model.fit(data_bag, target_bag)) # store fitted models here
        if self.oob:
            self.data = data
            self.target = target
        
    def predict(self, data):
        '''
        Get average prediction for every object from passed dataset
        '''
        # Your code here
        prediction = np.zeros(len(self.data))
        for idx, obj in enumerate(self.data):
            # Пробегаем по всем объектам исходной выборки. 
            # Затем пробегаемся по всем моделям и смотрим, какую метку для данного объекта (obj) предсказывает каждая модель.
            object_prediction_list = [model.predict([obj]) for model in self.models_list]
            # Сохраняем для данного объекта предсказания всех моделей. Итоговое предсказание для данного объекта -- среднее от предсказаний всех моделей.
            prediction[idx] = np.mean(object_prediction_list)
        return prediction
    
    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        # Your Code Here
        for i in range(len(self.data)):
            # Пробегаем по всем объектам исходной выборки. Для выбранного объекта пробегаем по всем бэгам.
            for idx, bag in enumerate(self.indices_list):
                # если в текущем бэге данного объекта нет, значит соответствующая можель на нем не обучалась.
                # Делаем предсказание для этого объекта на модели соответсвуюзего бэга.
                if i not in bag:
                    list_of_predictions_lists[i].append(self.models_list[idx].predict( [self.data[i]] ).item())
        # Таким образом, для каждого объекта мы получаем множество предсказаний тех моделей, которые его не видели.
        
        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)
    
    def _get_averaged_oob_predictions(self):
        '''
        Compute average prediction for every object from training set.
        If object has been used in all bags on training phase, return None instead of prediction
        '''
        self._get_oob_predictions_from_every_model()
        # Таким образом, для каждого объекта мы получаем множество предсказаний тех моделей, которые его не видели.
        # Теперь для каждого объекта усредняем данные предсказания, получая, таким образом, среднее предсказание.
        #! В идеальном мире, для каждого объекта найдется такая модель, которая на нем не обучалась.
        #! Но, на практике, у существуют объекты, которые попали в тренировочный сет КАЖОЙ модели.
        #! Для таких объектов список предсказаний на этом моменте программы будет пуст. Усреднение от пустого списка дает NaN.
        self.oob_predictions = [np.mean(np.array(pred)) for pred in self.list_of_predictions_lists] # Your Code Here
        
        
    def OOB_score(self):
        '''
        Compute mean square error for all objects, which have at least one prediction
        '''
        self._get_averaged_oob_predictions()
        # Для каждого объекта мы получили среднее предсказание от всех моделей. 
        # Теперь, поскольку мы знаем реальные метки этих объектов, просто посчитаем ошибку.
        #! Поскольку при усреднении предсказаний для каждого объекта были объекты, которые попали в обучающие выборки всех моделей
        #! в self.oob_predictions могут быть элементы NaN. Эти элементы не учитываются при усреднении, поэтому в генератор закладываем if-фильтр.
        return np.mean([item for item in (self.target - self.oob_predictions)**2 if not np.isnan(item)]) # Your Code Here

### Local tests:

In [7]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

#### Simple tests:

In [10]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Simple tests done!')

  0%|          | 0/100 [00:00<?, ?it/s]/home/pavel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pavel/.local/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  1%|          | 1/100 [00:01<02:03,  1.25s/it]/home/pavel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pavel/.local/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
  2%|▏         | 2/100 [00:02<02:01,  1.24s/it]/home/pavel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pavel/.local/lib/p

Simple tests done!


#### Medium tests

In [11]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'
    
print('Medium tests done!')

100%|██████████| 10/10 [00:03<00:00,  2.74it/s]

Medium tests done!


#### Complex tests:

In [12]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'
    
print('Complex tests done!')

100%|██████████| 10/10 [01:59<00:00, 11.91s/it]

Complex tests done!


In [13]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

0.0005505588285577012

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!